# 📝 Customer Segmentation README Data Generator

This notebook will generate all the tables and metrics needed for the README/dashboard mockup:

RFM Snapshot

KMeans Cluster Summary

Agglomerative Cluster Summary

Top Products per Segment

Example Customer Recommendations

Recommendation System Metrics

# 1️⃣ Setup & Imports

In [1]:
# Basic libraries
import pandas as pd
import numpy as np
from datetime import timedelta

# For clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering

# Optional: display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# 2️⃣ Load Data

In [2]:
# Load your cleaned transaction dataset
# Make sure df has: InvoiceNo, CustomerID, Description, Quantity, UnitPrice, InvoiceDate
df = pd.read_excel("OnlineRetail.xlsx")  # replace with your path

# Compute TotalPrice
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

# Optional sanity check
print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  TotalPrice  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom       15.30  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom       20.34  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom       22.00  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom       20.34  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom       20.34  


# 3️⃣ Compute RFM Metrics

In [3]:
# Snapshot date: 1 day after the last purchase
snapshot_date = df["InvoiceDate"].max() + timedelta(days=1)

# Compute RFM
rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (snapshot_date - x.max()).days,  # Recency
    "InvoiceNo": "nunique",                                   # Frequency
    "TotalPrice": "sum"                                       # Monetary
}).reset_index()

rfm.columns = ["CustomerID", "Recency", "Frequency", "Monetary"]

print("RFM Snapshot:\n", rfm.head())

RFM Snapshot:
    CustomerID  Recency  Frequency  Monetary
0     12346.0      326          2      0.00
1     12347.0        2          7   4310.00
2     12348.0       75          4   1797.24
3     12349.0       19          1   1757.55
4     12350.0      310          1    334.40


# 4️⃣ Scale RFM & Cluster Customers

In [4]:
# Scale RFM metrics
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[["Recency","Frequency","Monetary"]])

# --- KMeans Clustering ---
kmeans = KMeans(n_clusters=4, random_state=42)
rfm["Cluster_KMeans"] = kmeans.fit_predict(rfm_scaled)

# --- Agglomerative Clustering ---
agglo = AgglomerativeClustering(n_clusters=4)
rfm["Cluster_Agglo"] = agglo.fit_predict(rfm_scaled)

print("Clusters added to RFM data.\n", rfm.head())

C:\Users\dvenk\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:131: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\dvenk\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 247, in _count_physical_cores
    cpu_count_physical = _count_physical_cores_win32()
  File "C:\Users\dvenk\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 299, in _count_physical_cores_win32
    cpu_info = subprocess.run(
  File "C:\Users\dvenk\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 501, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\dven

Clusters added to RFM data.
    CustomerID  Recency  Frequency  Monetary  Cluster_KMeans  Cluster_Agglo
0     12346.0      326          2      0.00               1              2
1     12347.0        2          7   4310.00               3              0
2     12348.0       75          4   1797.24               3              0
3     12349.0       19          1   1757.55               3              0
4     12350.0      310          1    334.40               1              2


# 5️⃣ Cluster Summary Tables

In [5]:
# KMeans Cluster Summary
cluster_summary_kmeans = rfm.groupby("Cluster_KMeans").agg({
    "Recency": "mean",
    "Frequency": "mean",
    "Monetary": "mean",
    "CustomerID": "count"
}).rename(columns={"CustomerID": "NumCustomers"}).round(2)

# Agglomerative Cluster Summary
cluster_summary_agglo = rfm.groupby("Cluster_Agglo").agg({
    "Recency": "mean",
    "Frequency": "mean",
    "Monetary": "mean",
    "CustomerID": "count"
}).rename(columns={"CustomerID": "NumCustomers"}).round(2)

print("📊 K-Means Cluster Summary:\n", cluster_summary_kmeans)
print("\n📊 Agglomerative Cluster Summary:\n", cluster_summary_agglo)

📊 K-Means Cluster Summary:
                 Recency  Frequency   Monetary  NumCustomers
Cluster_KMeans                                             
0                 10.75      28.51   12168.26           194
1                248.93       1.81     455.11          1077
2                  5.09     109.91  124312.31            11
3                 42.78       4.37    1320.98          3090

📊 Agglomerative Cluster Summary:
                Recency  Frequency   Monetary  NumCustomers
Cluster_Agglo                                             
0                48.49       5.35    1709.83          3483
1                 6.73      85.00   56018.83            22
2               270.13       1.74     450.11           864
3                 3.67      64.67  241136.56             3


# 6️⃣ Assign Segment Names 

In [6]:
# Example mapping (adjust based on your analysis)
cluster_map = {
    0: "Frequent Buyers",
    1: "At-risk / Inactive",
    2: "High-value Loyal",
    3: "Price-sensitive"
}

rfm["Segment"] = rfm["Cluster_KMeans"].map(cluster_map)
print(rfm[["CustomerID","Recency","Frequency","Monetary","Segment"]].head())

   CustomerID  Recency  Frequency  Monetary             Segment
0     12346.0      326          2      0.00  At-risk / Inactive
1     12347.0        2          7   4310.00     Price-sensitive
2     12348.0       75          4   1797.24     Price-sensitive
3     12349.0       19          1   1757.55     Price-sensitive
4     12350.0      310          1    334.40  At-risk / Inactive


# 7️⃣ Top Products per Segment

In [7]:
# Merge segment info into transaction df
df = df.merge(rfm[['CustomerID', 'Segment']], on='CustomerID', how='left')

# Aggregate product quantity per segment
top_products_per_segment = (
    df.groupby(['Segment','Description'])['Quantity']
      .sum()
      .reset_index()
      .sort_values(['Segment','Quantity'], ascending=[True,False])
)

# Get top 5 per segment
top_products_summary = top_products_per_segment.groupby('Segment').head(5)

# Display nicely
for seg in top_products_summary['Segment'].unique():
    print(f"\nSegment: {seg}")
    temp = top_products_summary[top_products_summary['Segment']==seg]
    for i, row in enumerate(temp.itertuples(),1):
        print(f"{i}. {row.Description} ({row.Quantity})")


Segment: At-risk / Inactive
1. WORLD WAR 2 GLIDERS ASSTD DESIGNS (5377)
2. SMALL POPCORN HOLDER (4963)
3. WHITE HANGING HEART T-LIGHT HOLDER (4127)
4. FAIRY CAKE FLANNEL ASSORTED COLOUR (3320)
5. ASSORTED COLOURS SILK FAN (2624)

Segment: Frequent Buyers
1. WORLD WAR 2 GLIDERS ASSTD DESIGNS (24493)
2. JUMBO BAG RED RETROSPOT (20556)
3. POPCORN HOLDER (19059)
4. PACK OF 12 LONDON TISSUES  (15379)
5. ASSORTED COLOUR BIRD ORNAMENT (13732)

Segment: High-value Loyal
1. PACK OF 72 RETROSPOT CAKE CASES (6773)
2. RABBIT NIGHT LIGHT (6532)
3. WHITE HANGING HEART T-LIGHT HOLDER (6471)
4. SPACEBOY LUNCH BOX  (6215)
5. HEART OF WICKER SMALL (6200)

Segment: Price-sensitive
1. WORLD WAR 2 GLIDERS ASSTD DESIGNS (22097)
2. JUMBO BAG RED RETROSPOT (18190)
3. ASSORTED COLOUR BIRD ORNAMENT (17286)
4. PACK OF 72 RETROSPOT CAKE CASES (16645)
5. WHITE HANGING HEART T-LIGHT HOLDER (13546)


# 8️⃣ Personalized Recommendations per CustomerID

In [10]:
# ===============================
# 8️⃣ Personalized Recommendations per CustomerID (Fixed NaN & SettingWithCopyWarning)
# ===============================
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import random

# --- Ensure your df has columns: CustomerID, Description, Quantity ---
# df = pd.read_excel("online_retail_cleaned.xlsx")  # Load your dataset if not already loaded

# 1️⃣ Remove rows with missing CustomerID and make a copy
df_clean = df.dropna(subset=['CustomerID']).copy()

# 2️⃣ Convert CustomerID to integer
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)

# 3️⃣ Create customer-product pivot table
customer_product_matrix = df_clean.pivot_table(
    index='CustomerID', 
    columns='Description', 
    values='Quantity', 
    aggfunc='sum', 
    fill_value=0
)

# 4️⃣ Compute customer similarity (cosine similarity)
customer_similarity = cosine_similarity(customer_product_matrix)
customer_similarity_df = pd.DataFrame(
    customer_similarity, 
    index=customer_product_matrix.index, 
    columns=customer_product_matrix.index
)

# 5️⃣ Function to get top N recommended products for a given customer
def get_top_n_recommendations(customer_id, top_n=5):
    if customer_id not in customer_similarity_df.index:
        return []
    
    # Similar customers excluding self
    similar_customers = customer_similarity_df.loc[customer_id].sort_values(ascending=False).index[1:]
    
    # Aggregate products purchased by similar customers
    rec_products = df_clean[df_clean['CustomerID'].isin(similar_customers)].groupby('Description')['Quantity'].sum()
    
    # Remove products already purchased by this customer
    purchased_products = df_clean[df_clean['CustomerID']==customer_id]['Description'].unique()
    rec_products = rec_products[~rec_products.index.isin(purchased_products)]
    
    # Top N recommendations
    top_recs = rec_products.sort_values(ascending=False).head(top_n)
    return list(top_recs.index)

# 6️⃣ Generate recommendations for all customers
recommendations = {cid: get_top_n_recommendations(cid, top_n=5) 
                   for cid in df_clean['CustomerID'].unique()}

# 7️⃣ Example: Display recommendations for 3 random customers
for cid in random.sample(list(df_clean['CustomerID'].unique()), 3):
    print(f"\nCustomerID: {cid}")
    recs = recommendations.get(cid, [])
    for i, prod in enumerate(recs, 1):
        print(f"{i}. {prod}")


CustomerID: 12931
1. WORLD WAR 2 GLIDERS ASSTD DESIGNS
2. JUMBO BAG RED RETROSPOT
3. WHITE HANGING HEART T-LIGHT HOLDER
4. PACK OF 72 RETROSPOT CAKE CASES
5. MINI PAINT SET VINTAGE 

CustomerID: 16110
1. WORLD WAR 2 GLIDERS ASSTD DESIGNS
2. JUMBO BAG RED RETROSPOT
3. ASSORTED COLOUR BIRD ORNAMENT
4. WHITE HANGING HEART T-LIGHT HOLDER
5. POPCORN HOLDER

CustomerID: 15571
1. WORLD WAR 2 GLIDERS ASSTD DESIGNS
2. ASSORTED COLOUR BIRD ORNAMENT
3. WHITE HANGING HEART T-LIGHT HOLDER
4. PACK OF 72 RETROSPOT CAKE CASES
5. POPCORN HOLDER
